In [2]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
import time
import sqlite3

In [3]:
class JobScraper:
    def __init__(self, db_name='jobs.db'):
        self.db_name = db_name
        self.conn = sqlite3.connect(self.db_name)
        self.cursor = self.conn.cursor()
        self.create_table()
    
    def drop_table(self):
        self.cursor.execute('DROP TABLE IF EXISTS Job')
        self.conn.commit()

    def create_table(self):
        self.drop_table()
        self.cursor.execute('''
        CREATE TABLE Job (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT,
            salary TEXT,
            location TEXT
        )
        ''')
        self.conn.commit()

    def insert_job(self, name, salary, location):
        self.cursor.execute('''
        INSERT INTO Job (name, salary, location)
        VALUES (?, ?, ?)
        ''', (name, salary, location))
        self.conn.commit()

    def close_db(self):
        self.conn.close()

    def fetch_jobs(self, location):
        self.cursor.execute('SELECT name, salary FROM Job WHERE location=?', (location,))
        return self.cursor.fetchall()

    def scrape_jobs(self, url, prefecture_code, city_code, location_name):
        driver = webdriver.Chrome()
        driver.get(url)
        time.sleep(1)

        driver.find_element(By.CLASS_NAME, "retrieval_icn").click()
        time.sleep(1)

        element = driver.find_element(By.ID, "ID_tDFK1CmbBox")
        Select(element).select_by_value(prefecture_code)
        time.sleep(1)

        driver.find_element(By.ID, "ID_Btn").click()
        time.sleep(1)

        element = driver.find_element(By.ID, "ID_rank1CodeMulti")
        Select(element).select_by_value(city_code)
        time.sleep(1)

        driver.find_element(By.ID, "ID_ok").click()
        time.sleep(1)

        driver.find_element(By.ID, "ID_searchBtn").click()
        time.sleep(1)

        soup = BeautifulSoup(driver.page_source, "html.parser")
        jobs = soup.find_all("table", attrs={"class": "kyujin"})[1::]

        for job in jobs:
            job_name = str(job.find("td", attrs={"class": "m13"}).text.strip())
            salary_tags = job.find_all("tr", attrs={"class": "border_new"})[5].select(".disp_inline_block")
            job_salary = ""
            for salary_tag in salary_tags:
                job_salary = salary_tag.text
            self.insert_job(job_name, job_salary, location_name)

        driver.close()